Improvise from https://www.datacamp.com/community/tutorials/autoencoder-keras-tutorial
for Deep Hybrid Network with GAN

Good rpactice:
1. keras_inception = pandas, flow from pandas dataframe

In [20]:
import keras
from keras_gan_rgb import GAN
from matplotlib import pyplot as plt
import numpy as np
import gzip
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Dense, Flatten, Dropout, Concatenate, concatenate
from keras.models import Model
from keras.utils import to_categorical, plot_model
from keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split, GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier, BaseWrapper
from sklearn.metrics import classification_report

%matplotlib inline

In [6]:
def extract_data(filename, num_images):
    with gzip.open(filename) as bytestream:
        bytestream.read(16)
        buf = bytestream.read(28 * 28 * num_images)
        data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
        data = data.reshape(num_images, 28,28)
        return data

In [7]:
def extract_labels(filename, num_images):
    with gzip.open(filename) as bytestream:
        bytestream.read(8)
        buf = bytestream.read(1 * num_images)
        labels = np.frombuffer(buf, dtype=np.uint8).astype(np.int64)
        return labels

In [9]:
train_data = extract_data('data_notMNIST-to-MNIST/train-images-idx3-ubyte.gz', 60000)
test_data = extract_data('data_notMNIST-to-MNIST/t10k-images-idx3-ubyte.gz', 10000)

train_labels = extract_labels('data_notMNIST-to-MNIST/train-labels-idx1-ubyte.gz',60000)
test_labels = extract_labels('data_notMNIST-to-MNIST/t10k-labels-idx1-ubyte.gz',10000)

In [10]:
# Create dictionary of target classes
label_dict = {
 0: 'A',
 1: 'B',
 2: 'C',
 3: 'D',
 4: 'E',
 5: 'F',
 6: 'G',
 7: 'H',
 8: 'I',
 9: 'J',
}

In [ ]:
"""
# TODO: random plot
plt.figure(figsize=[5,5])

# Display the first image in training data
plt.subplot(121)
curr_img = np.reshape(train_data[9], (28,28))
curr_lbl = train_labels[0]
plt.imshow(curr_img, cmap='gray')
plt.title("(Label: " + str(label_dict[curr_lbl]) + ")")

# Display the first image in testing data
plt.subplot(122)
curr_img = np.reshape(test_data[9], (28,28))
curr_lbl = test_labels[0]
plt.imshow(curr_img, cmap='gray')
plt.title("(Label: " + str(label_dict[curr_lbl]) + ")")
"""

In [11]:
# Original train_data = (None, 28, 28) --> channel last
train_data = train_data.reshape(-1, 28,28, 1)
test_data = test_data.reshape(-1, 28,28, 1)
train_data.shape, test_data.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [12]:
train_data = train_data / np.max(train_data)
test_data = test_data / np.max(test_data)

In [13]:
# D : (train_data, train_data, train_labels) -> (train_X,valid_X,train_labels,valid_labels)
train_X,valid_X, train_ground, valid_ground, train_labels, valid_labels = train_test_split(train_data, 
                                                                                          train_data, 
                                                                                          train_labels, 
                                                                                          test_size=0.2,
                                                                                          random_state=13,
                                                                                          stratify=train_labels)

In [15]:
batch_size = 128
epochs = 10
# inChannel = 1
# x, y = 28, 28

input_shape = train_X[0,0:].shape
losses = {"D_out": "categorical_crossentropy", "G_out": "mse"}
alpha = [10**0, 10**1, 10**2, 10**3, 10**4, 10**5]
beta  = [1., 2., 3., 4., 5.]
loss_weights = {"D_out": alpha, "G_out": beta}

In [ ]:
# define parameterised model (function/class)
model_name = "dhm_autoencoder"

def dhm_autoencoder(opt='sgd',input_shape=input_shape, losses=losses, plotModel=True):
    # construct, compile and return a Keras model
    
    # ==================
    # autoencoder
    # ==================
    #encoder
    #input = 28 x 28 x 1 (wide and thin)
    input_img = Input(shape =input_shape, name="Input")
    #
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img) #28 x 28 x 32
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1) #14 x 14 x 32
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1) #14 x 14 x 64
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2) #7 x 7 x 64
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2) #7 x 7 x 128 (small and thick)
    #decoder
    conv4 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3) #7 x 7 x 128
    up1 = UpSampling2D((2,2))(conv4) # 14 x 14 x 128
    conv5 = Conv2D(64, (3, 3), activation='relu', padding='same')(up1) # 14 x 14 x 64
    up2 = UpSampling2D((2,2))(conv5) # 28 x 28 x 64
    G_out = Conv2D(1, (3, 3), activation='sigmoid', padding='same', name="G_out")(up2) # 28 x 28 x 1
    #
    # ==================
    # discriminator
    # ==================
    conv1_d  = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img) #28 x 28 x 32
    pool1_d  = MaxPooling2D(pool_size=(2, 2))(conv1_d) #14 x 14 x 32
    conv2_d  = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1_d) #14 x 14 x 64
    pool2_d  = MaxPooling2D(pool_size=(2, 2))(conv2_d) #7 x 7 x 64
    conv3_d  = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2_d) #7 x 7 x 128 (small and thick)
    #
    merge    = concatenate([conv3_d, conv3])
    pool3_d  = MaxPooling2D(pool_size=(2, 2))(merge)
    dropout1 = Dropout(0.25)(pool3_d)
    #
    flatten  = Flatten()(pool3_d)
    dense    = Dense(128, activation='relu')(flatten)
    dropout2 = Dropout(0.5)(dense)
    D_out    = Dense(10, activation='softmax', name="D_out")(dropout2)
    #
    
    model    = Model(inputs=[input_img], outputs=[D_out, G_out])
    model    = model.compile(optimizer=opt, losses=losses , metrics='accuracy',)
    #
    if plotModel == True:
        plot_model(model, to_file=model_name+".png", show_shapes=True)
    #
    return model

In [ ]:
class KerasClassifier_ext(BaseWrapper):
    """Implementation of the scikit-learn classifier API for Keras.
    """

    def fit(self, x, y, sample_weight=None, **kwargs):
        y = np.array(y)
        if len(y.shape) == 2 and y.shape[1] > 1:
            self.classes_ = np.arange(y.shape[1])
        elif (len(y.shape) == 2 and y.shape[1] == 1) or len(y.shape) == 1:
            self.classes_ = np.unique(y)
            y = np.searchsorted(self.classes_, y)
        else:
            raise ValueError('Invalid shape for y: ' + str(y.shape))
        self.n_classes_ = len(self.classes_)
        if sample_weight is not None:
            kwargs['sample_weight'] = sample_weight
        return super(KerasClassifier_ext, self).fit(x, y, **kwargs)

    def predict(self, x, **kwargs):
        kwargs = self.filter_sk_params(Sequential.predict_classes, kwargs)

        proba = self.model.predict(x, **kwargs)
        if proba.shape[-1] > 1:
            classes = proba.argmax(axis=-1)
        else:
            classes = (proba > 0.5).astype('int32')
        return self.classes_[classes]

    def predict_proba(self, x, **kwargs):
        kwargs = self.filter_sk_params(Sequential.predict_proba, kwargs)
        probs = self.model.predict(x, **kwargs)

        # check if binary classification
        if probs.shape[1] == 1:
            # first column is probability of class 0 and second is of class 1
            probs = np.hstack([1 - probs, probs])
        return probs

    def score(self, x, y, **kwargs):
        y = np.searchsorted(self.classes_, y)
        kwargs = self.filter_sk_params(Sequential.evaluate, kwargs)

        loss_name = self.model.loss
        if hasattr(loss_name, '__name__'):
            loss_name = loss_name.__name__
        if loss_name == 'categorical_crossentropy' and len(y.shape) != 2:
            y = to_categorical(y)

        outputs = self.model.evaluate(x, y, **kwargs)
        outputs = to_list(outputs)
        for name, output in zip(self.model.metrics_names, outputs):
            if name == 'acc':
                return output
        raise ValueError('The model is not configured to compute accuracy. '
                         'You should pass `metrics=["accuracy"]` to '
                         'the `model.compile()` method.')

In [16]:
# multiple output layers in Scikit-Learn wrappers
# Source : https://github.com/keras-team/keras/issues/9001
from sklearn.base import BaseEstimator

class KerasModel(BaseEstimator):
    def __init__(self, optimizer = 'sgd', losses = losses, plotModel = True):
        self.optimizer = optimizer # an example of a tunable hyperparam
        self.losses = losses
        self.plotModel = True
    # in GridSearchCV, expect 'fit' and 'predict' method
    # use BaseExtimator to generalize 
    def fit(self, X, y):  # 'fit' means, 'fit' a 'KerasModel'
        #
        # from original code; X is Dictionary
        # make 'fit' receive dictionanry of 'y'
        input_img = Input(shape=input_shape, name="Input")
        #
        conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img) #28 x 28 x 32
        pool1 = MaxPooling2D(pool_size=(2, 2))(conv1) #14 x 14 x 32
        conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1) #14 x 14 x 64
        pool2 = MaxPooling2D(pool_size=(2, 2))(conv2) #7 x 7 x 64
        conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2) #7 x 7 x 128 (small and thick)
        #decoder
        conv4 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3) #7 x 7 x 128
        up1 = UpSampling2D((2,2))(conv4) # 14 x 14 x 128
        conv5 = Conv2D(64, (3, 3), activation='relu', padding='same')(up1) # 14 x 14 x 64
        up2 = UpSampling2D((2,2))(conv5) # 28 x 28 x 64
        G_out = Conv2D(1, (3, 3), activation='sigmoid', padding='same', name="G_out")(up2) # 28 x 28 x 1
        # ==================
        # discriminator
        # ==================
        conv1_d  = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img) #28 x 28 x 32
        pool1_d  = MaxPooling2D(pool_size=(2, 2))(conv1_d) #14 x 14 x 32
        conv2_d  = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1_d) #14 x 14 x 64
        pool2_d  = MaxPooling2D(pool_size=(2, 2))(conv2_d) #7 x 7 x 64
        conv3_d  = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2_d) #7 x 7 x 128 (small and thick)
        #
        merge    = concatenate([conv3_d, conv3])
        pool3_d  = MaxPooling2D(pool_size=(2, 2))(merge)
        dropout1 = Dropout(0.25)(pool3_d)
        #
        flatten  = Flatten()(pool3_d)
        dense    = Dense(128, activation='relu')(flatten)
        dropout2 = Dropout(0.5)(dense)
        D_out    = Dense(10, activation='softmax', name="D_out")(dropout2)
        #
        if self.plotModel == True:
            plot_model(model, to_file=model_name+".png", show_shapes=True)  # make it into self
        ###
        self.model = Model(inputs = input_img, outputs = [D_out, G_out])
        self.model.compile(self.optimizer, 'mse', losses=losses, metrics='accuracy')
        # call fit model
        self.model.fit(X, y)
        return self

    def predict(self, X):
        return self.model.predict(X)

# End

In [18]:
from sklearn.pipeline import make_pipeline
dhm = make_pipeline(KerasModel())

In [22]:
# build 
# dhm = dhm_autoencoder(x, y, inChannel, opt, losses, alpha, beta, plot_model=True)
# model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# dhm_autoencoder() / KerasModel()
dhm = KerasClassifier(build_fn=KerasModel, 
                      batch_size=batch_size, 
                      epochs=epochs, 
                      verbose=0)

In [23]:
# gridsearch for single I/O
grid = GridSearchCV(estimator=dhm, param_grid=loss_weights, n_jobs=-1)

In [24]:
# train_X.shape, train_labels.reshape((train_labels.shape[0], 1)).shape, train_ground.shape

In [25]:
train_y = {"D_out": to_categorical(train_labels.reshape((train_labels.shape[0], 1)).shape), "G_out" : train_ground}
# train_y = {"D_out": to_categorical(train_labels.reshape((train_labels.shape[0], 1)).shape)}

grid_result = grid.fit(train_X, train_y)

ValueError: Found input variables with inconsistent numbers of samples: [48000, 2]

In [ ]:
# use scikit-learn pipeline
dhm_train = dhm.fit(train_X, {"D_out": to_categorical(train_labels), "G_out" : train_ground}, 
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(valid_X, {"D_out": to_categorical(valid_labels), "G_out" : valid_ground}))

In [ ]:
loss = dhm_train.history['D_out_loss']
val_loss = dhm_train.history['val_loss']

np.savetxt(model_name+"_D_out_loss.txt", np.array(loss), delimiter=",")
np.savetxt(model_name+"_D_out_val_loss.txt", np.array(val_loss), delimiter=",")

epochs_ = range(epochs)
plt.figure()
plt.plot(epochs_, loss, 'bo', label='Training loss')
plt.plot(epochs_, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
loss = dhm_train.history['G_out_loss']
val_loss = dhm_train.history['val_loss']

np.savetxt(model_name+"_G_out_loss.txt", np.array(loss), delimiter=",")
np.savetxt(model_name+"_G_out_val_loss.txt", np.array(val_loss), delimiter=",")

epochs_ = range(epochs)
plt.figure()
plt.plot(epochs_, loss, 'bo', label='Training loss')
plt.plot(epochs_, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
[pred_D, pred_G] = dhm.predict(test_data)

In [ ]:
pred_D.shape, pred_G.shape

In [ ]:
# pred_D analysis: confsion matrix, top-2 accuracy etc.
# use sklearn.metrics.classification_report()

In [ ]:
# pred_G analysis:
plt.figure(figsize=(20, 4))
print("Test Images")
for i in range(10):
    plt.subplot(2, 10, i+1)
    plt.imshow(test_data[i, ..., 0], cmap='gray')
    curr_lbl = test_labels[i]
    plt.title("(Label: " + str(label_dict[curr_lbl]) + ")")
plt.show()    
plt.figure(figsize=(20, 4))
print("Reconstruction of Test Images")
for i in range(10):
    plt.subplot(2, 10, i+1)
    plt.imshow(pred_G[i, ..., 0], cmap='gray')  
plt.show()